In [1]:
import pandas as pd

In [2]:
csv_filename = './../cleaned_database/movies.csv'
df = pd.read_csv(csv_filename)

C:\Users\RohanDalvi\AppData\Local\Temp\ipykernel_12616\348238252.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_filename)


In [3]:
def parse_list(cell):
    if pd.isnull(cell) or cell == "":
        return []
    return [item.strip() for item in cell.split(", ")]

In [ ]:
movie_data_dict = {}

for _, row in df.iterrows():
    movie_id = row["Movie ID"]
    movie_data_dict[movie_id] = {
        "name": row["Name"],
        "date": row["Date"],
        "tagline": row["Tagline"],
        "description": row["Description"],
        "minute": row["Duration"], 
        "rating": row["Rating"],
        "actors": parse_list(row["Actors"]),
        "roles": parse_list(row["Roles"]),
        "crew": parse_list(row["Crew"]),
        "work": parse_list(row["Work"]),
        "countries": parse_list(row["Countries"]),
        "genres": parse_list(row["Genres"]),
        "languages": parse_list(row["Languages"]),
        "types": parse_list(row["Types"]),
        "posters": parse_list(row["Posters"]),
        "studios": parse_list(row["Studios"]),
        "themes": parse_list(row["Themes"])
    }

In [5]:
first_5_keys = list(movie_data_dict.keys())[:5]  
for key in first_5_keys:
    print(f"Movie ID: {key}, Data: {movie_data_dict[key]}")


Movie ID: 1000001, Data: {'name': 'Barbie', 'date': 2023.0, 'tagline': "She's everything. He's just Ken.", 'description': 'Barbie and Ken are having the time of their lives in the colorful and seemingly perfect world of Barbie Land. However, when they get a chance to go to the real world, they soon discover the joys and perils of living among humans.', 'minute': 114.0, 'rating': 3.86, 'actors': ['Margot Robbie', 'Ryan Gosling', 'America Ferrera', 'Ariana Greenblatt', 'Issa Rae', 'Kate McKinnon', 'Alexandra Shipp', 'Emma Mackey', 'Hari Nef', 'Sharon Rooney', 'Ana Cruz Kayne', 'Ritu Arya', 'Dua Lipa', 'Nicola Coughlan', 'Emerald Fennell', 'Simu Liu', 'Kingsley Ben-Adir', 'Ncuti Gatwa', 'Scott Evans', 'John Cena', 'Michael Cera', 'Rhea Perlman', 'Helen Mirren', 'Will Ferrell', 'Connor Swindells', 'Jamie Demetriou', 'Andrew Leung', 'Will Merrick', 'Zheng Xi Yong', 'Asim Chaudhry', 'Ray Fearon', 'Erica Ford', 'Hannah Khalique-Brown', 'Mette', 'Marisa Abela', 'Lucy Boynton', 'Rob Brydon', 'T

In [6]:
!pip install chromadb

In [6]:
import chromadb
chroma_client = chromadb.PersistentClient(path="./../chromadb_client")


In [8]:
chroma_client.delete_collection(name="movies_database")

ValueError: Collection movies_database does not exist.

In [7]:
collection = chroma_client.create_collection(name="movies_database")


In [12]:
def insert_movies_to_chroma(movie_data_dict):
    documents = []
    metadatas = []
    ids = []
    separator = " , "


    for movie_id, movie_data in movie_data_dict.items():
        
        joint_actors_string = separator.join(movie_data.get("actors", []))
        joint_roles_string = separator.join(movie_data.get("roles", []))
        joint_crew_string = separator.join(movie_data.get("crew", []))
        joint_genres_string = separator.join(movie_data.get("genres", []))
        joint_countries_string = separator.join(movie_data.get("countries", []))
        joint_languages_string = separator.join(movie_data.get("languages", []))
        joint_studios_string = separator.join(movie_data.get("studios", []))
        joint_themes_string = separator.join(movie_data.get("themes", []))

        document = f"{movie_data['name']} | {movie_data['description']} | {joint_actors_string} | {joint_roles_string} | {joint_crew_string} | {joint_genres_string} | {joint_countries_string} | {joint_languages_string} | {joint_studios_string} | {joint_themes_string}"
        metadata = {
            "date": movie_data.get("date"),
            "tagline": movie_data.get("tagline"),
            "minute": movie_data.get("minute"),
            "rating": movie_data.get("rating"),
            "actors": joint_actors_string,
            "roles": joint_roles_string,
            "crew": joint_crew_string,
            "work": separator.join(movie_data.get("work", [])),
            "countries": joint_countries_string,
            "genres": joint_genres_string,
            "languages": joint_languages_string,
            "types": separator.join(movie_data.get("types", [])),
            "posters": separator.join(movie_data.get("posters", [])),
            "studios": joint_studios_string,
            "themes": joint_themes_string
        }
        documents.append(document)
        metadatas.append(metadata)
        ids.append(str(movie_id))

    collection.add(documents=documents, metadatas=metadatas, ids=ids)


In [9]:
print(movie_data_dict[1000001])

{'name': 'Barbie', 'date': 2023.0, 'tagline': "She's everything. He's just Ken.", 'description': 'Barbie and Ken are having the time of their lives in the colorful and seemingly perfect world of Barbie Land. However, when they get a chance to go to the real world, they soon discover the joys and perils of living among humans.', 'minute': 114.0, 'rating': 3.86, 'actors': ['Margot Robbie', 'Ryan Gosling', 'America Ferrera', 'Ariana Greenblatt', 'Issa Rae', 'Kate McKinnon', 'Alexandra Shipp', 'Emma Mackey', 'Hari Nef', 'Sharon Rooney', 'Ana Cruz Kayne', 'Ritu Arya', 'Dua Lipa', 'Nicola Coughlan', 'Emerald Fennell', 'Simu Liu', 'Kingsley Ben-Adir', 'Ncuti Gatwa', 'Scott Evans', 'John Cena', 'Michael Cera', 'Rhea Perlman', 'Helen Mirren', 'Will Ferrell', 'Connor Swindells', 'Jamie Demetriou', 'Andrew Leung', 'Will Merrick', 'Zheng Xi Yong', 'Asim Chaudhry', 'Ray Fearon', 'Erica Ford', 'Hannah Khalique-Brown', 'Mette', 'Marisa Abela', 'Lucy Boynton', 'Rob Brydon', 'Tom Stourton', 'Chris Tayl

In [ ]:

def insert_first_1000(movie_data_dict):
    """Inserts the first 1000 items from the dictionary."""

    first_1000 = {}
    keys = list(movie_data_dict.keys())

    for i in range(min(1000, len(keys))): 
        print(keys[i])
        first_1000[keys[i]] = movie_data_dict[keys[i]]
        
    insert_movies_to_chroma(first_1000)

In [ ]:

def insert_first_10000(movie_data_dict):
    """Inserts the first 10000 items from the dictionary."""

    first_10000 = {}
    keys = list(movie_data_dict.keys())

    for i in range(min(10000, len(keys))):
        print(keys[i])
        first_10000[keys[i]] = movie_data_dict[keys[i]]
        
    insert_movies_to_chroma(first_10000)

In [13]:
insert_first_1000(movie_data_dict)

1000001
1000002
1000003
1000004
1000005
1000006
1000007
1000008
1000009
1000010
1000011
1000012
1000013
1000014
1000015
1000016
1000017
1000018
1000019
1000020
1000021
1000022
1000023
1000024
1000025
1000026
1000027
1000028
1000029
1000030
1000031
1000032
1000033
1000034
1000035
1000036
1000037
1000038
1000039
1000040
1000041
1000042
1000043
1000044
1000045
1000046
1000047
1000048
1000049
1000050
1000051
1000052
1000053
1000054
1000055
1000056
1000057
1000058
1000059
1000060
1000061
1000062
1000063
1000064
1000065
1000066
1000067
1000068
1000069
1000070
1000071
1000072
1000073
1000074
1000075
1000076
1000077
1000078
1000079
1000080
1000081
1000082
1000083
1000084
1000085
1000086
1000087
1000088
1000089
1000090
1000091
1000092
1000093
1000094
1000095
1000096
1000097
1000098
1000099
1000100
1000101
1000102
1000103
1000104
1000105
1000106
1000107
1000108
1000109
1000110
1000111
1000112
1000113
1000114
1000115
1000116
1000117
1000118
1000119
1000120
1000121
1000122
1000123
1000124
1000125


In [ ]:
def query_chroma_and_print_first(query_text, filter_criteria=None):
    
    try:
        if filter_criteria:
            results = collection.query(
                query_texts=[query_text],
                n_results=1,  
                where=filter_criteria,
            )
        else:
            results = collection.query(
                query_texts=[query_text],
                n_results=1,  
            )

        if results["documents"] and results["documents"][0]:
            print("Closest Result:")
            print("Document:", results["documents"][0][0])
            if results["metadatas"] and results["metadatas"][0]:
                print("Metadata:", results["metadatas"][0][0]) 
            else:
                print("No metadata found.")

        else:
            print("No results found.")

    except Exception as e:
        print(f"An error occurred: {e}")



In [15]:
print("Sample Documents:")
print(collection.peek())

Sample Documents:
{'ids': ['1000001', '1000002', '1000003', '1000004', '1000005', '1000006', '1000007', '1000008', '1000009', '1000010'], 'embeddings': array([[ 0.02774008, -0.07351816,  0.01096311, ..., -0.07372393,
         0.01552615,  0.02190758],
       [-0.04738233,  0.073709  ,  0.04274208, ..., -0.02370867,
        -0.03434431,  0.0355713 ],
       [-0.00914809, -0.09818979,  0.01430484, ..., -0.02696304,
        -0.04745173, -0.02138913],
       ...,
       [ 0.00079269, -0.0546625 , -0.08344812, ..., -0.01960568,
        -0.00332915, -0.05201688],
       [-0.01464594, -0.07022114, -0.04084756, ..., -0.11587565,
        -0.01076085, -0.01954553],
       [-0.0446317 , -0.11134337, -0.03962792, ..., -0.03392874,
         0.09495831, -0.02954918]]), 'documents': ["Barbie | Barbie and Ken are having the time of their lives in the colorful and seemingly perfect world of Barbie Land. However, when they get a chance to go to the real world, they soon discover the joys and perils of l

In [ ]:

query_chroma_and_print_first("movies about friendships")



Closest Result:
Document: The Edge of Seventeen | Two high school girls are best friends until one dates the other's older brother, who is totally his sister's nemesis. | Hailee Steinfeld , Woody Harrelson , Haley Lu Richardson , Blake Jenner , Kyra Sedgwick , Hayden Szeto , Alexander Calvert , Eric Keenleyside , Nesta Cooper , Meredith Monroe , Katie Stuart , Lina Renna , Christian Michael Cooper , Ava Grace Cooper , Laine MacNeil , Jena Skodje , Daniel Bacon , Kavandeep Hayre , Lyle Reginald , Chris Shields , Christian Lagasse , Lauren K. Robek , Laura Ward | Nadine Franklin , Mr. Bruner , Krista , Darian Franklin , Mona Franklin , Erwin Kim , Nick Mossman , Tom Franklin , Shannon , Greer , Jeannie , Little Nadine , Little Darian , Little Krista , TCBY Girl , Little Mean Girl , Film Festival Emcee , Teammate , Junior Guy , Hospital Official , Usher , Krista's Mom , Erwin's Friend | Kelly Fremon Craig , Kelly Fremon Craig , Richard Sakai , Julie Ansell , Amy Brooks , James L. Brooks ,

In [ ]:

query_chroma_and_print_first(
    "comedy movies with high ratings", filter_criteria={ "rating": {"$gt": 4.5}}
)



Closest Result:
Document: Over the Garden Wall | Two brothers, Wirt and Greg, find themselves lost in the Unknown; a strange forest adrift in time. With the help of a wise old Woodsman and a foul-tempered bluebird named Beatrice, Wirt and Greg must travel across this strange land, in hope of finding their way home. Join them as they encounter surprises and obstacles on their journey through the wood. | Elijah Wood , Collin Dean | Wirt (voice) , Gregory (voice) | Nate Cash , Pernelle Hayes , Curtis Lelash , Patrick McHale , Brian A. Miller , Jennifer Pelphrey , Rob Sorcher , Nick Cross , Nate Cash , Josh Kaufman , Justin Rubenstein , Alex Borquez , Timothy J. Borquez , Nick Gotten III | Animation , Mystery , Family , Comedy | USA | English | Warner Bros. Television , Cartoon Network Studios | Fairy-tale fantasy and enchanted magic , Kids' animated fun and adventure , Emotional and captivating fantasy storytelling , Dreamlike , quirky , and surreal storytelling , Holiday joy and heartwar

In [ ]:

query_chroma_and_print_first(
    "i hate my life and want to die"
)

Closest Result:
Document: The Virgin Suicides | A group of male friends become obsessed with five mysterious sisters who are sheltered by their strict, religious parents. | James Woods , Kathleen Turner , Kirsten Dunst , Josh Hartnett , Michael ParÃ© , Scott Glenn , Danny DeVito , A.J. Cook , Hanna Hall , Leslie Hayman , Chelse Swain , Anthony DeSimone , Lee Kagan , Robert Schwartzman , Noah "40" Shebib , Jonathan Tucker , Joe Roncetti , Hayden Christensen , Chris Hale , Joe Dinicol , Suki Kaiser , Dawn Greenhalgh , Allen Stewart-Coates , Sherry Miller , Jonathan Whittaker , MichÃ¨le Duquet , Murray McRae , Roberta Hanley , Paul Sybersma , Susan Sybersma , Peter Snider , Gary Brennan , Charles Boyland , Dustin Ladd , Kristin Fairlie , Melody Johnson , Sheyla Molho , Ashley Ainsworth , Courtney Hawkrigg , FranÃ§ois Klanfer , Mackenzie Lawrenz , Tim Hall , Amos Crawley , Andrew Gillies , Mairlyn Smith , Sally Cahill , Tracey Ferencz , Scot Denton , Catherine Swing , Timothy Adams , Micha

In [ ]:

query_chroma_and_print_first(
    "action movie with peter parker"
)

Closest Result:
Document: The Amazing Spider-Man 2 | For Peter Parker, life is busy. Between taking out the bad guys as Spider-Man and spending time with the person he loves, Gwen Stacy, high school graduation cannot come quickly enough. Peter has not forgotten about the promise he made to Gwenâs father to protect her by staying away, but that is a promise he cannot keep. Things will change for Peter when a new villain, Electro, emerges, an old friend, Harry Osborn, returns, and Peter uncovers new clues about his past. | Andrew Garfield , Emma Stone , Jamie Foxx , Dane DeHaan , Colm Feore , Felicity Jones , Paul Giamatti , Sally Field , Embeth Davidtz , Campbell Scott , Marton Csokas , Louis Cancelmi , Max Charles , B.J. Novak , Sarah Gadon , Michael Massee , Jorge Vega , Bill Heck , Teddy Coluca , Helen Stern , Aidy Bryant , Cal McCrystal , Anslem Richardson , Mark Doherty , James Colby , Kari Coleman , Skyler Gisondo , Charlie DePew , Robert Newman , Adrian Martinez , Thaddeus Phil

In [ ]:

query_chroma_and_print_first(
    "war movie with sad ending"
)

Closest Result:
Document: Fury | In the last months of World War II, as the Allies make their final push in the European theatre, a battle-hardened U.S. Army sergeant named 'Wardaddy' commands a Sherman tank called 'Fury' and its five-man crew on a deadly mission behind enemy lines. Outnumbered and outgunned, Wardaddy and his men face overwhelming odds in their heroic attempts to strike at the heart of Nazi Germany. | Brad Pitt , Shia LaBeouf , Logan Lerman , Michael PeÃ±a , Jon Bernthal , Jim Parrack , Brad William Henke , Kevin Vance , Xavier Samuel , Jason Isaacs , Anamaria Marinca , Alicia von Rittberg , Scott Eastwood , Laurence Spellman , Daniel Betts , Adam Ganne , Eric Kofi Abrefa , Osi Okerafor , John Macmillan , Saul Barrett , Marek Oravec , Kyle Soller , Jake Curran , Jack Bannon , Branko TomoviÄ , Orion Lee , Vivien Bridson , Christian Contreras , Stella Stocker , Jacob Vonhendial , Lukas Rolfe , Leon Rolfe , Harry Hancock , Daniel Dorr , Bernhard Forcher , Edin Gali , Jai

In [15]:
chroma_client = chromadb.PersistentClient(path="./../chromadb_client")
collection = chroma_client.get_collection(name="movies_database")